In [1]:
import pandas as pd 
import numpy as np
import ast
from datetime import date

from sklearn.neighbors import KNeighborsRegressor

# Extraction date
date=str(date.today())
date = '2020-11-03'

In [2]:
#####################################
# Load data 
#####################################
# Read
df = pd.read_csv(f'extractions/extraction_{date}.csv', dtype={'propertyCode':str})

#####################################
# Transform data 
#####################################
# Filter outliers
df = df[df['size'] < 50000]
df['floor'] = df['floor'].map(lambda x: '7' if x==x and x.isdigit() and int(x)>7 else x)
df['floor'] = df['floor'].map(lambda x: 'st' if x=='-1' else x)
df['floor'] = df['floor'].map(lambda x: 'st' if x=='ss' else x)
df['floor'] = df['floor'].map(lambda x: 'bj' if x=='en' else x)
df = df[~df.propertyType.isin(['studio','chalet'])]
df['status'] = df['status'].map(lambda x: 'good' if x=='newdevelopment' else x)

# Normalise numeric regressors
for c in ['price','size','priceByArea']:
    df[c] = np.log(df[c].values + 1)

# Create desired variables 
df['type'] = df['detailedType'].map(lambda x: dict(ast.literal_eval(x))['typology'])

df['parkingSpace'] = df['parkingSpace'].fillna('''{'hasParkingSpace': False, 'isParkingSpaceIncludedInPrice': False}''')
df['hasParkingSpace'] = df['parkingSpace'].map(lambda x: dict(ast.literal_eval(x))['hasParkingSpace'])
df['isParkingSpaceIncludedInPrice'] = df['parkingSpace'].map(lambda x: dict(ast.literal_eval(x))['isParkingSpaceIncludedInPrice'])

df['status'] = df['status'].fillna('missing')
df['floor'] = df['floor'].fillna(df['floor'].mode()[0])
df['hasLift'] = df['hasLift'].fillna(df['hasLift'].mode()[0])

# Distance in feature space from nearest neighbours from longitude and latitude
neigh = KNeighborsRegressor(n_neighbors=5)
neigh.fit(df[['latitude','longitude']].values, df['priceByArea'])
df['neighbourhood_estimated_priceByArea'] = neigh.predict(df[['latitude','longitude']].values)

# Booleans to categorical
df.dtypes
cat_cols = df.columns[df.dtypes.isin(['bool','object'])]
for c in cat_cols:
    df[c] = df[c].astype(str)

# Drop unneeded variables
drop_cols = ['Unnamed: 0', 'index', 'address', 'country','detailedType','distance','externalReference','latitude','longitude','municipality','province','suggestedTexts','thumbnail','url','operation','parkingSpace','type','district','neighborhood','newDevelopmentFinished', 'newDevelopment', 'hasStaging']
drop_cols.append('priceByArea')
df.drop(columns=drop_cols, inplace=True, errors='ignore')

# Drop duplicate index and keep propertycode as index
df.drop_duplicates(inplace=True)
df = df.set_index('propertyCode')

df.to_parquet(f'transformed/extraction_{date}.parquet')

In [3]:
df.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
bathrooms,842,NaN,NaN,NaN,1.09145,0.296549,1,1,1,1,3
exterior,842,2,True,438,NaN,NaN,NaN,NaN,NaN,NaN,NaN
floor,842,9,bj,228,NaN,NaN,NaN,NaN,NaN,NaN,NaN
has360,842,2,False,734,NaN,NaN,NaN,NaN,NaN,NaN,NaN
has3DTour,842,2,False,775,NaN,NaN,NaN,NaN,NaN,NaN,NaN
hasLift,842,2,True,573,NaN,NaN,NaN,NaN,NaN,NaN,NaN
hasPlan,842,2,True,527,NaN,NaN,NaN,NaN,NaN,NaN,NaN
hasVideo,842,2,False,626,NaN,NaN,NaN,NaN,NaN,NaN,NaN
numPhotos,842,NaN,NaN,NaN,24.1793,10.7442,0,17,24,30,91
price,842,NaN,NaN,NaN,12.4321,0.264132,11.5522,12.2368,12.4875,12.6519,12.7657


In [4]:
(df == 'True').sum()

bathrooms                              0
exterior                               0
floor                                  0
has360                                 0
has3DTour                              0
hasLift                                0
hasPlan                                0
hasVideo                               0
numPhotos                              0
price                                  0
propertyType                           0
rooms                                  0
showAddress                            0
size                                   0
status                                 0
topNewDevelopment                      0
hasParkingSpace                        0
isParkingSpaceIncludedInPrice          0
neighbourhood_estimated_priceByArea    0
dtype: int64